Time Series Forecast with NHiTS on the Vierlinden dataset (all sensors, 2021)

In [1]:
import sys
sys.path.append('./pytorch-forecasting/')
import pandas as pd
import numpy as np
import torch
import lightning.pytorch as pl
from pytorch_forecasting import NHiTS, TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
pl.seed_everything(42)

Global seed set to 42


42

### Load dataset

In [2]:
# Read the dataset into a DataFrame
data = pd.read_csv('./RIWWER/Vierlinden/Vierlinden_2021_All.csv')

# Drop columns that have lots of missing values
data.drop(["FLP_Hohenstand_Pumpensumpf_pval","FLP_Strom_P3_pval","FLP_Strom_P4_pval","FLP_Strom_P5_pval","Durchfluss SWP1 und SWP2_pval","FLP_Hohenstand_Becken1_pval","FLP_Hohenstand_Becken3_pval","FLP_Hohenstand_Beckne2_pval"], axis=1, inplace=True)

# NaNs are not allowed by the model
data.fillna(method="bfill", inplace=True)
data.fillna(method="ffill", inplace=True)

# Set Datetime as index
data['Datetime'] = pd.to_datetime(data['Datetime'])

# One time series for the whole year?
data['series'] = 0

# As many timesteps per timeseries as hours in every month?
time_idx = []
for i in range(1):
    timesteps = len( data[ data['series'] == i ] )
    time_idx += list(range(timesteps))
data['time_idx'] = time_idx

In [3]:
data.head()

,Datetime,Schieber Position_pval,Oberwasser_pval,Unterwasser_pval,Durchflumenge_pval,Berechnete Durchflussmenge_pval,Fllstand SWS_pval,Fllstand RWS_pval,Strom P1_pval,Strom P2_pval,...,Strom Pumpe 2_pval1,Strom Pumpe 3_pval,Niederschlag,Füllstand_RRB,Entleerung_RüB,Füllstand_RüB_1,Füllstand_RüB_2,Füllstand_RüB_3,series,time_idx
0,2021-01-01 00:00:00,100.0,8.140845,5.753623,7.689189,7.732558,75.717949,36.0,1.076923,0.0,...,0.0,1.0,0.000000,1.47,0.098,3.16,3.08,2.72,0,0
1,2021-01-01 01:00:00,100.0,8.000000,5.173913,6.808219,8.271739,75.717949,36.0,1.076923,0.0,...,0.0,1.0,1.182353,1.47,0.099,3.16,3.08,2.72,0,1
2,2021-01-01 02:00:00,100.0,7.967742,5.000000,5.813333,7.197674,75.717949,36.0,1.076923,0.0,...,0.0,1.0,1.182353,1.47,0.096,3.16,3.08,2.72,0,2
3,2021-01-01 03:00:00,100.0,7.076923,4.843750,4.216216,4.743243,75.717949,36.0,1.076923,0.0,...,0.0,1.0,1.182353,1.47,0.098,3.16,3.08,2.72,0,3
4,2021-01-01 04:00:00,100.0,8.464789,5.466667,8.384615,8.325000,75.717949,36.0,1.076923,0.0,...,0.0,1.0,1.182353,1.47,0.098,3.16,3.08,2.72,0,4


In [4]:
# Parameters for dataloaders
max_encoder_length = 24*2
max_prediction_length = 5*2
training_cutoff = data["time_idx"].max() * 4 // 5 # 80% for training
context_length = max_encoder_length
prediction_length = max_prediction_length
batch_size = 32

In [5]:
# Load best model (from NHits_Vierlinden_Train.ipynb)
best_model_path = './RIWWER/torch_forecasting/model_checkpoints/NHits_Vierlinden/lightning_logs/version_0/checkpoints/epoch=14-step=2250.ckpt'
best_model = NHiTS.load_from_checkpoint(best_model_path)

/home/tchiaburu/anaconda3/envs/torch_gpu_riwwer_new/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:196: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
/home/tchiaburu/anaconda3/envs/torch_gpu_riwwer_new/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:196: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(


### Corrupt sensor clusters

In [6]:
fraction = 1.0   # fraction of rows to corrupt
sampling = 'NAR' # sampling mechanism for corruptions, options are completely at random ('CAR'),
                 # at random ('AR'), not at random ('NAR')

# Sensor clusters
# Weather and rain tanks: ['Niederschlag', 'Füllstand_RRB', 'Füllstand_RüB_1', 'Füllstand_RüB_2', 'Füllstand_RüB_3']
sensors = {
    "Herzogstr":     ['Schieber Position_pval', 'Oberwasser_pval', 'Unterwasser_pval', 'Durchflumenge_pval', 'Berechnete Durchflussmenge_pval'],
    #"Kaiserstr":     ['Fllstand SWS_pval', 'Fllstand RWS_pval', 'Strom P1_pval', 'Strom P2_pval', 'Strom P3_pval', 'Strom P4_pval', 'Strom P5_pval', 'Strom P6_pval'],
    #"Kreuzweg":      ['Fllstand Pumpensumpf_pval', 'Strom Pumpe 1_pval', 'Strom Pumpe 2_pval'],
    #"Vierlindenhof": ['Fllstand Pumpensumpf_pval1', 'Strom Pumpe 1_pval1', 'Strom Pumpe 2_pval1', 'Strom Pumpe 3_pval']
}

In [7]:
# Turn off sensors iteratively according to the cluster location
for cluster in sensors:
    print("\n\n++++++++++ Turning off " + cluster + " ++++++++++\n")

    # Copy dataset (we want to investigate what impact the turning off of individual clusters has)
    data_copy = data.copy(deep=True)

    ### Sample rows and corrupt them ###
    for column in sensors[cluster]:
        if fraction == 1.0:
            rows = data_copy.index
        # Completely At Random
        # elif sampling == 'CAR':
        #     rows = np.random.permutation(data_copy.index)[:int(len(data_copy)*fraction)]
        # elif sampling == 'NAR' or sampling == 'AR':
        #     n_values_to_discard = int(len(data_copy) * min(fraction, 1.0))
        #     perc_lower_start = np.random.randint(0, len(data_copy) - n_values_to_discard)
        #     perc_idx = range(perc_lower_start, perc_lower_start + n_values_to_discard)
        #     # Not At Random
        #     if sampling == 'NAR':
        #         # pick a random percentile of values in this column
        #         rows = data_copy[column].sort_values().iloc[perc_idx].index
        #     # At Random
        #     elif sampling == 'AR':
        #         depends_on_col = np.random.choice(list(set(data_copy.columns) - {column}))
        #         # pick a random percentile of values in other column
        #         rows = data_copy[depends_on_col].sort_values().iloc[perc_idx].index
        # Overwrite values with 0 (random value better?)
        #data_copy.loc[rows, column] = 0.0

    ### Create Dataloader for corrupted dataset ###
    validation = TimeSeriesDataSet(
        data_copy[lambda x: x.time_idx > training_cutoff],
        target_normalizer="auto",
        time_idx="time_idx",
        target="Entleerung_RüB",
        categorical_encoders={"series": NaNLabelEncoder().fit(data_copy.series)},
        group_ids=["series"],
        time_varying_unknown_reals=list(set(data_copy.columns) - {'Datetime', 'series', 'time_idx'}),
        max_encoder_length=context_length,
        min_encoder_length=max_encoder_length,
        max_prediction_length=prediction_length,
        min_prediction_length=max_prediction_length,
        allow_missing_timesteps=True
    )
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=18)

    ### Make prediction and evaluate ###
    actuals = torch.cat([y[0] for x, y in iter(val_dataloader)]).to(torch.device('cuda:0'))
    predictions = best_model.predict(val_dataloader,
                                     trainer_kwargs=dict(default_root_dir="./RIWWER/torch_forecasting/model_checkpoints/NHits_Vierlinden"))
    err = actuals - predictions
    mae = err.abs().mean()
    print('MAE = ' + str(mae))
    rmse = torch.sqrt( torch.square(err).mean() )
    print('RMSE = ' + str(rmse))



++++++++++ Turning off Herzogstr ++++++++++



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


MAE = tensor(2.5372, device='cuda:0')
RMSE = tensor(10.4089, device='cuda:0')


In [8]:
# Note: Columns are permuted every time!
# Also note that some of the values will already be predominantly 0; not to be confounded with your own corrupted zeros
for x, y in iter(val_dataloader):
    print(x['encoder_cont'][0][0])

tensor([-0.5253, -0.8876, -0.2080,  0.3277, -0.3542,  0.0000, -0.3168, -0.0343,
        -0.3695,  0.0000, -0.5737,  0.0000, -0.1662, -0.5797, -0.4267,  0.2865,
         0.0484,  0.0000,  0.1159, -1.0347, -0.6172,  2.5225, -0.5339, -0.1855,
         0.0000, -0.4248])
tensor([ 0.4267, -0.7687, -0.2080,  0.3277, -0.3542,  0.0000, -0.3168, -0.0134,
         0.4557,  0.0000, -0.5737,  0.0000, -0.1542,  0.7125, -0.4908, -0.2818,
         0.0484,  0.0000,  0.1318,  0.1016, -0.0537,  2.5225,  0.3815, -0.1855,
         0.0000,  0.0588])
tensor([ 0.6659, -1.2627, -0.2080,  0.3277, -0.3542,  0.0000, -0.3168, -1.6022,
         0.7742,  0.0000, -0.5737,  0.0000, -0.3221,  7.6081, -0.4006, -0.2232,
        -1.6573,  0.0000, -1.5710, -0.2915,  0.2703,  2.5225,  0.2223, -0.1855,
         0.0000, -0.2989])
tensor([-0.6438,  0.3677, -0.2080,  0.3277, -0.3542,  0.0000, -0.3168, -0.6824,
        -1.4598,  0.0000, -0.5737,  0.0000, -0.4060, -0.5797, -0.4231,  2.4826,
        -0.2440,  0.0000, -0.4252, -0.1

In [16]:
val_dataloader.__dict__

{'dataset': TimeSeriesDataSet[length=1695](
 	time_idx='time_idx',
 	target='Entleerung_RüB',
 	group_ids=['series'],
 	weight=None,
 	max_encoder_length=48,
 	min_encoder_length=48,
 	min_prediction_idx=7008,
 	min_prediction_length=10,
 	max_prediction_length=10,
 	static_categoricals=[],
 	static_reals=[],
 	time_varying_known_categoricals=[],
 	time_varying_known_reals=[],
 	time_varying_unknown_categoricals=[],
 	time_varying_unknown_reals=['Unterwasser_pval', 'Fllstand Pumpensumpf_pval1', 'Strom P2_pval', 'Schieber Position_pval', 'Strom Pumpe 2_pval', 'Strom P6_pval', 'Strom Pumpe 1_pval', 'Füllstand_RRB', 'Fllstand Pumpensumpf_pval', 'Strom P5_pval', 'Strom Pumpe 1_pval1', 'Strom Pumpe 3_pval', 'Füllstand_RüB_3', 'Niederschlag', 'Fllstand SWS_pval', 'Entleerung_RüB', 'Füllstand_RüB_1', 'Strom Pumpe 2_pval1', 'Füllstand_RüB_2', 'Strom P1_pval', 'Durchflumenge_pval', 'Fllstand RWS_pval', 'Oberwasser_pval', 'Strom P3_pval', 'Strom P4_pval', 'Berechnete Durchflussmenge_pval'],
 	va

In [10]:
validation.save('./RIWWER/Vierlinden/val_dataset')

In [11]:
loaded_validation = validation.load('./RIWWER/Vierlinden/val_dataset')

In [12]:
loaded_val_dataloader = loaded_validation.to_dataloader(train=False, batch_size=batch_size, num_workers=18)

In [13]:
for x, y in iter(loaded_val_dataloader):
    print(x['encoder_cont'][0][0])

tensor([-0.5253, -0.8876, -0.2080,  0.3277, -0.3542,  0.0000, -0.3168, -0.0343,
        -0.3695,  0.0000, -0.5737,  0.0000, -0.1662, -0.5797, -0.4267,  0.2865,
         0.0484,  0.0000,  0.1159, -1.0347, -0.6172,  2.5225, -0.5339, -0.1855,
         0.0000, -0.4248])
tensor([ 0.4267, -0.7687, -0.2080,  0.3277, -0.3542,  0.0000, -0.3168, -0.0134,
         0.4557,  0.0000, -0.5737,  0.0000, -0.1542,  0.7125, -0.4908, -0.2818,
         0.0484,  0.0000,  0.1318,  0.1016, -0.0537,  2.5225,  0.3815, -0.1855,
         0.0000,  0.0588])
tensor([ 0.6659, -1.2627, -0.2080,  0.3277, -0.3542,  0.0000, -0.3168, -1.6022,
         0.7742,  0.0000, -0.5737,  0.0000, -0.3221,  7.6081, -0.4006, -0.2232,
        -1.6573,  0.0000, -1.5710, -0.2915,  0.2703,  2.5225,  0.2223, -0.1855,
         0.0000, -0.2989])
tensor([-0.6438,  0.3677, -0.2080,  0.3277, -0.3542,  0.0000, -0.3168, -0.6824,
        -1.4598,  0.0000, -0.5737,  0.0000, -0.4060, -0.5797, -0.4231,  2.4826,
        -0.2440,  0.0000, -0.4252, -0.1

In [17]:
loaded_val_dataloader.__dict__

{'dataset': TimeSeriesDataSet[length=1695](
 	time_idx='time_idx',
 	target='Entleerung_RüB',
 	group_ids=['series'],
 	weight=None,
 	max_encoder_length=48,
 	min_encoder_length=48,
 	min_prediction_idx=7008,
 	min_prediction_length=10,
 	max_prediction_length=10,
 	static_categoricals=[],
 	static_reals=[],
 	time_varying_known_categoricals=[],
 	time_varying_known_reals=[],
 	time_varying_unknown_categoricals=[],
 	time_varying_unknown_reals=['Unterwasser_pval', 'Fllstand Pumpensumpf_pval1', 'Strom P2_pval', 'Schieber Position_pval', 'Strom Pumpe 2_pval', 'Strom P6_pval', 'Strom Pumpe 1_pval', 'Füllstand_RRB', 'Fllstand Pumpensumpf_pval', 'Strom P5_pval', 'Strom Pumpe 1_pval1', 'Strom Pumpe 3_pval', 'Füllstand_RüB_3', 'Niederschlag', 'Fllstand SWS_pval', 'Entleerung_RüB', 'Füllstand_RüB_1', 'Strom Pumpe 2_pval1', 'Füllstand_RüB_2', 'Strom P1_pval', 'Durchflumenge_pval', 'Fllstand RWS_pval', 'Oberwasser_pval', 'Strom P3_pval', 'Strom P4_pval', 'Berechnete Durchflussmenge_pval'],
 	va

In [22]:
# Note: After replacing the feature with zeros, the TimeSeriesDataset applies 'transform_values'
#       which leads to all the values in that column being replaced not by 0, but by another rescaled value
loaded_validation.set_overwrite_values(0.0, 'Unterwasser_pval', 'all')
loaded_val_dataloader_corr = loaded_validation.to_dataloader(train=False, batch_size=batch_size, num_workers=18)
for x, y in iter(loaded_val_dataloader_corr):
    print(x['encoder_cont'][0][0])

/home/tchiaburu/anaconda3/envs/torch_gpu_riwwer_new/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


tensor([-2.9263, -0.8876, -0.2080,  0.3277, -0.3542,  0.0000, -0.3168, -0.0343,
        -0.3695,  0.0000, -0.5737,  0.0000, -0.1662, -0.5797, -0.4267,  0.2865,
         0.0484,  0.0000,  0.1159, -1.0347, -0.6172,  2.5225, -0.5339, -0.1855,
         0.0000, -0.4248])
tensor([-2.9263, -0.7687, -0.2080,  0.3277, -0.3542,  0.0000, -0.3168, -0.0134,
         0.4557,  0.0000, -0.5737,  0.0000, -0.1542,  0.7125, -0.4908, -0.2818,
         0.0484,  0.0000,  0.1318,  0.1016, -0.0537,  2.5225,  0.3815, -0.1855,
         0.0000,  0.0588])
tensor([-2.9263, -1.2627, -0.2080,  0.3277, -0.3542,  0.0000, -0.3168, -1.6022,
         0.7742,  0.0000, -0.5737,  0.0000, -0.3221,  7.6081, -0.4006, -0.2232,
        -1.6573,  0.0000, -1.5710, -0.2915,  0.2703,  2.5225,  0.2223, -0.1855,
         0.0000, -0.2989])
tensor([-2.9263,  0.3677, -0.2080,  0.3277, -0.3542,  0.0000, -0.3168, -0.6824,
        -1.4598,  0.0000, -0.5737,  0.0000, -0.4060, -0.5797, -0.4231,  2.4826,
        -0.2440,  0.0000, -0.4252, -0.1

In [24]:
loaded_validation._overwrite_values

{'values': tensor(-2.9263, dtype=torch.float64),
 'variable': 'Unterwasser_pval',
 'target': 'all'}